# Home task: Collaborative filtering

In [27]:
import numpy as np
import pandas as pd

In [28]:
def J(Y, R, X, Theta, lambd):
    '''
    params: 1d vector  of X and Theta
    :return expression for cost function
    '''
    assert (X.shape[1] == Theta.shape[0])
    h = X @ Theta

    try:
        assert (h.shape == Y.shape)
    except:
        print('h.shape {} !=Y.shape {}'.format(h.shape, Y.shape))

    J = 1 / 2 * np.sum(((h - Y) * R) ** 2) + lambd / 2 * np.sum(X ** 2) + lambd / 2 * np.sum(Theta ** 2)
    try:
        assert (len(J.shape) == 0)
    except:
        print('J is not raw number. J.shape = ', J.shape)

    return J

In [29]:
def J_derivative(Y,R,X, Theta, lambd): 
   
    cost_matr= (X @ Theta - Y) * R #  n_movies * n_users 

    X_grad= cost_matr @ Theta.T 
    Theta_grad= (cost_matr.T @ X).T
    
   
    try: 
        assert(X_grad.shape ==X.shape)
        assert(Theta_grad.shape ==Theta.shape)
    except:
        print ('Check gradient calculus')

    # Regularization part :          
    X_grad  += lambd * X
    Theta_grad += lambd * Theta

    
    return X_grad, Theta_grad

In [30]:
def fit(Y, R, num_features=10, alpha=0.0001, lambd=.01, eps=.1, max_iter=1000, step=100, verbose=0):
    num_movies, num_users = Y.shape

    if verbose:
        print('Running gradient descent with alpha= {}, lambda= {}, eps= {}, max_iter= {}'.format(
            alpha, lambd, eps, max_iter))

    #     X= params[:num_movies*num_features].reshape(num_movies,num_features)
    #     Theta = params[num_movies*num_features:].reshape(num_features,num_users)

    np.random.seed(2019)
    X = np.random.randn(num_movies, num_features)
    Theta = np.random.randn(num_features, num_users)

    J_hist = [-1]  # used for keeping J values. Init with -1 to avoid 0 at first iter
    continue_iter = True  # flag to continue next iter (grad desc step)
    iter_number = 0  # used for limit by max_iter

    try:
        while continue_iter:
            # Do step of gradient descent
            X_grad, Theta_grad = J_derivative(Y, R, X, Theta, lambd)
            X = X - alpha * X_grad
            Theta = Theta - alpha * Theta_grad

            # keep history of J values
            J_hist.append(J(Y, R, X, Theta, lambd))
            # check criteria of exit (finish grad desc)
            if iter_number > max_iter:  # if limit succeeded
                continue_iter = False
                print('iter_number> max_iter')
            elif np.abs(J_hist[iter_number - 1] - J_hist[iter_number]) < eps:  # if accuracy is succeeded
                continue_iter = False
                print('J_hist[iter_number]={}'.format(J_hist[iter_number]))
            iter_number += 1

            if verbose and iter_number % step == 0:
                print('{}: {}'.format(iter_number, J_hist[iter_number - 1]))
        return X, Theta, J_hist
    except Exception as e:
        print('Training is interrupted due to error:', e)
        return X, Theta, J_hist

In [31]:
def normalize_Y(Y,R, n_0):
    Ymean = np.zeros((n_0, 1))
    Ynorm = np.zeros(Y.shape)
    for i in range(n_0):
        idx = R[i,:] == 1
        Ymean[i] = Y[i, idx].mean()
        Ynorm[i, idx]= Y[i, idx] - Ymean[i]
    return Ymean, Ynorm

In [32]:
def fit_collaborative_filtering(Y, R, n_features=20, max_iter=50000, verbose=1, return_J_hist= False):
    '''
        Y: df of provided values
        R: df of 0 and 1 - marked values as provided (e.g. R is 1 for elements of Y that are not 0)
    '''

    scale = Y.max() - Y.min()
    Y_scaled = Y / scale * 10
    n_0 = Y_scaled.shape[0]
    Ymean, Ynorm = normalize_Y(Y_scaled.values, R.values, n_0)
    X, Theta, J_hist = fit(Ynorm, R.values, num_features=n_features, alpha=0.0005, lambd=1, max_iter=max_iter,
                           eps=.01, step=50, verbose=verbose)
    # if verbose:
    #     draw_cost_changes(J_hist)

    pred = X @ Theta
    pred_rescaled = (pred + Ymean) * scale.values / 10

    df_results_pivot= pd.DataFrame(pred_rescaled , index= Y.index, columns = Y.columns)

    if return_J_hist:
        return df_results_pivot, J_hist
    else:
        return df_results_pivot

In [33]:
def convert_to_matrix(df, index, columns, values):
    '''
        e.g. values='average_msv', index='phrase', columns='locode'
    '''
    df_target=df.pivot_table(index=index, columns=columns, values=values, aggfunc=np.max, dropna= False)
    return df_target

In [34]:
def fit_target(df_target, index, columns, values, n_features=20, max_iter=5000, verbose=1):
    Y= df_target.fillna(0) # not sure it is necessary
    R= df_target.applymap(lambda x: 0 if np.isnan(x) else 1)

    df_results_pivot, J_hist = fit_collaborative_filtering(Y, R, n_features=n_features, max_iter=max_iter, verbose=verbose, return_J_hist= True)

    df_results_pivot_temp= pd.DataFrame(df_results_pivot.to_records())
    df_results = pd.melt(df_results_pivot_temp,
                id_vars=index, # 'iid',
                value_vars=list(df_results_pivot_temp.columns[1:]),
                var_name= columns,# 'uid',
                value_name= '{}_pred'.format(values)) # 'rating_pred')
    return df_results,  J_hist

In [35]:
def round_to_existing(val, existing_unique_values):
    return existing_unique_values[np.argmin(np.abs(existing_unique_values-val))]

In [36]:
def fill_missed(df_input, df_missed= None, compute_average_for_blank_columns= False, n_features=20, max_iter=5000, verbose=1, return_J_hist= False):
    '''
    :param df_input: df with three columns - index, columns, values- all values are provided
    :param df_missed- df with 2 or 3 columns - but first two are index, columns
    :return:  df_predict - that contains all possible combinations
    '''
    index, columns, values= list(df_input)
    print ('index: {}, columns: {}, values: {}'.format(index, columns, values))

    if compute_average_for_blank_columns:
        df_input_atleast= df_input
    else:
        df_input_atleast = df_input.dropna()

    df_target = convert_to_matrix(df_input_atleast, index=index, columns=columns, values=values)
    df_pred, J_hist = fit_target(df_target, index, columns, values, n_features=n_features, max_iter=max_iter, verbose=verbose)

    existing_unique_values = np.array(sorted(df_input[values].unique()))
    df_pred['{}_pred_round'.format(values)]= df_pred['{}_pred'.format(values)].apply(lambda x: round_to_existing(x,existing_unique_values))
    df_pred[index]=df_pred[index].astype(df_input.dtypes[index])
    df_pred[columns] = df_pred[columns].astype(df_input.dtypes[columns])
    df_pred= df_pred.merge(df_input, how='outer', on = ([index, columns]))
    df_pred= df_pred[[index,columns, values, '{}_pred_round'.format(values), '{}_pred'.format(values)]]

    if df_missed is None:
        df_out= df_pred
    else:
        df_out = df_missed.merge(df_pred, how= 'left', on= ([index, columns]))

    if return_J_hist:
        return df_out, J_hist
    else:
        return df_out

Loading amazon_reviews.csv

[Link](https://www.kaggle.com/code/saurav9786/recommender-system-using-amazon-reviews/input)

In [37]:
df = pd.read_csv('amazon_reviews.csv')
df.columns = ['ProductId', 'UserId', 'Rating', 'Timestamp']
# Limiting to 4000 products and 300 users
df = df.iloc[:4000,:300]
df.drop(['Timestamp'], axis=1, inplace=True)
df_pred = fill_missed(df, compute_average_for_blank_columns= False, n_features=20, max_iter=5000, return_J_hist= False)

index: ProductId, columns: UserId, values: Rating


C:\Users\sviat\AppData\Local\Temp\ipykernel_9660\4021017623.py:5: FutureWarning: The provided callable <function max at 0x00000179FEB7CEE0> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_target=df.pivot_table(index=index, columns=columns, values=values, aggfunc=np.max, dropna= False)
C:\Users\sviat\AppData\Local\Temp\ipykernel_9660\655695788.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  R= df_target.applymap(lambda x: 0 if np.isnan(x) else 1)


Running gradient descent with alpha= 0.0005, lambda= 1, eps= 0.01, max_iter= 5000
50: 40023.51136284431
100: 37227.39980963594
150: 35238.86759656685
200: 33467.40030270443
250: 31815.03847304077
300: 30254.890736156234
350: 28775.808349873958
400: 27371.29392675368
450: 26036.589433368135
500: 24767.74029929926
550: 23561.2420448521
600: 22413.887615520856
650: 21322.693097158033
700: 20284.857342859996
750: 19297.737587350806
800: 18358.83318177663
850: 17465.773741754863
900: 16616.309853381896
950: 15808.305358981981
1000: 15039.730682286392
1050: 14308.656881544266
1100: 13613.250243480636
1150: 12951.767301109074
1200: 12322.550199188787
1250: 11724.022355566807
1300: 11154.684381729683
1350: 10613.11023545581
1400: 10097.943584696952
1450: 9607.894365998065
1500: 9141.73552364507
1550: 8698.299917770168
1600: 8276.477391127157
1650: 7875.211985352602
1700: 7493.499298366749
1750: 7130.383975214024
1800: 6784.957325146455
1850: 6456.35505814881
1900: 6143.755134417529
1950: 5846.

In [38]:
df_pred

,ProductId,UserId,Rating,Rating_pred_round,Rating_pred
0,A00766851QZZUBOVF4JFT,0321732944,NaN,5.0,4.996172
1,A0293130VTX2ZXA70JQS,0321732944,NaN,5.0,5.007755
2,A030530627MK66BD8V4LN,0321732944,NaN,4.0,4.002613
3,A0590501PZ7HOWJKBGQ4,0321732944,NaN,5.0,5.007166
4,A076219533YHEV2LJO988,0321732944,NaN,3.0,3.025022
...,...,...,...,...,...
739211,AZTC8ZV20NO1D,6301977173,NaN,1.0,1.000042
739212,AZV9WA9MNT0FB,6301977173,NaN,5.0,4.999948
739213,AZYNQZ94U6VDB,6301977173,NaN,5.0,4.999955
739214,AZYTSU42BZ7TP,6301977173,NaN,3.0,3.000019


In [39]:
# Users who have rated the product
df_pred[df_pred['Rating'].notna()]

,ProductId,UserId,Rating,Rating_pred_round,Rating_pred
1408,A2CX7LUOHB2NDG,0321732944,5.0,5.0,5.000000
4424,A1GI0U4ZRJA8WN,0439886341,1.0,1.0,1.000000
5649,A2NWSAGRHCP8N5,0439886341,1.0,1.0,1.000000
5908,A2WNBOD3WNDNKT,0439886341,3.0,3.0,3.000000
8632,A1QGNMC6O1VW39,0511189877,5.0,5.0,5.000000
...,...,...,...,...,...
738840,ANQM9OUAX68P5,6301977173,2.0,2.0,1.999998
738956,ARM2TEKZ7I2S7,6301977173,5.0,5.0,4.999996
738979,ASG60YVY09J6D,6301977173,1.0,1.0,1.000002
739108,AWGROQYSH9KHJ,6301977173,5.0,5.0,5.000022
